In [2]:
from rdflib import Graph, Namespace, URIRef, Literal, OWL
from rdflib.namespace import RDF, RDFS
import sys

try:
    # Initialiser les graphes
    ontology_graph = Graph()
    dbpedia_graph = Graph()
    geonames_graph = Graph()
    integrated_graph = Graph()

    # Charger les fichiers
    print("Chargement des fichiers...")
    ontology_graph.parse("ontology.ttl", format="turtle")
    dbpedia_graph.parse("dbpedia_cities.ttl", format="turtle")
    geonames_graph.parse("geonames_data.ttl", format="turtle")

    # Ajouter l'ontologie
    print("Intégration de l'ontologie...")
    for triple in ontology_graph:
        integrated_graph.add(triple)

    # Définir les namespaces
    DBO = Namespace("http://dbpedia.org/ontology/")
    GN = Namespace("http://www.geonames.org/ontology#")
    
    # Lier les données
    print("Création des liens owl:sameAs...")
    def normalize_name(name):
        return name.lower().strip()

    count_links = 0
    for dbp_city in dbpedia_graph.subjects(RDF.type, DBO.City):
        dbp_name = str(dbpedia_graph.value(dbp_city, RDFS.label))
        for geo_city in geonames_graph.subjects(RDF.type, GN.City):
            geo_name = str(geonames_graph.value(geo_city, RDFS.label))
            if normalize_name(dbp_name) == normalize_name(geo_name):
                integrated_graph.add((dbp_city, OWL.sameAs, geo_city))
                count_links += 1

    # Ajouter toutes les données
    print("Intégration des données...")
    for triple in dbpedia_graph:
        integrated_graph.add(triple)
    for triple in geonames_graph:
        integrated_graph.add(triple)

    # Sauvegarder les résultats
    print("Sauvegarde des résultats...")
    integrated_graph.serialize("integrated_data2.ttl", format="turtle")
    integrated_graph.serialize("integrated_data2.rdf", format="xml")

    # Afficher les statistiques
    print(f"\nStatistiques:")
    print(f"Triples dans l'ontologie: {len(ontology_graph)}")
    print(f"Triples de DBpedia: {len(dbpedia_graph)}")
    print(f"Triples de GeoNames: {len(geonames_graph)}")
    print(f"Liens owl:sameAs créés: {count_links}")
    print(f"Total des triples intégrés: {len(integrated_graph)}")

except Exception as e:
    print(f"Erreur: {str(e)}", file=sys.stderr)
    sys.exit(1)

Chargement des fichiers...
Intégration de l'ontologie...
Création des liens owl:sameAs...
Intégration des données...
Sauvegarde des résultats...

Statistiques:
Triples dans l'ontologie: 22
Triples de DBpedia: 3883
Triples de GeoNames: 279
Liens owl:sameAs créés: 13
Total des triples intégrés: 4197
